## Query existing database

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
print(pd.__version__)

1.1.3


In [3]:
database_path = 'Resources/dow.sqlite'
engine = create_engine(f'sqlite:///{database_path}')

In [4]:
# method 1
data = engine.execute("SELECT * FROM dow")
print(type(data))

i = 0
for r in data:
    if i < 5:
        print(r)
        i += 1

<class 'sqlalchemy.engine.result.ResultProxy'>
(1, 1, 'AA', '2011-01-07', 15.82, 16.72, 15.78, 16.42, 239655616, None)
(2, 1, 'AA', '2011-01-14', 16.71, 16.71, 15.64, 15.97, 242963398, None)
(3, 1, 'AA', '2011-01-21', 16.19, 16.38, 15.6, 15.79, 138428495, None)
(4, 1, 'AA', '2011-01-28', 15.87, 16.63, 15.82, 16.13, 151379173, None)
(5, 1, 'AA', '2011-02-04', 16.18, 17.39, 16.18, 17.14, 154387761, None)


In [5]:
# method 2
conn = engine.connect()
df = pd.read_sql("SELECT * FROM dow", conn)
df.head()

,id,quarter,stock,date,open_price,high_price,low_price,close_price,volume,percent_change
0,1,1,AA,2011-01-07,15.82,16.72,15.78,16.42,239655616,None
1,2,1,AA,2011-01-14,16.71,16.71,15.64,15.97,242963398,None
2,3,1,AA,2011-01-21,16.19,16.38,15.60,15.79,138428495,None
3,4,1,AA,2011-01-28,15.87,16.63,15.82,16.13,151379173,None
4,5,1,AA,2011-02-04,16.18,17.39,16.18,17.14,154387761,None


## Create table

##### SQLAlchemy uses Python classes as its primary means to communicate and make changes to SQL databases
##### This makes SQLAlchemy an ORM because it uses objects to map changes to SQL tables/databases

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

In [7]:
# Create Rabbit closs (table)
class Pet(Base):
    __tablename__ = "pet"
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    color = Column(String(255))
    kind = Column(String(255))
    age = Column(Integer)

In [8]:
# Create Owl class (table)
class Robot(Base):
    __tablename__ = "robot"
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    origin = Column(String(255))


In [9]:
# Create database connection
database_path = 'Resources/pets_cafe.db'
engine = create_engine(f'sqlite:///{database_path}')


In [10]:
# Create Pet table and Robot table in the database
Base.metadata.create_all(engine)

# # Drop Table
# Base.metadata.drop_all(engine)

In [11]:
# Create instances for each class
rabbit = Pet(name='mimi', color='white', kind='rabbit', age=2)
rabbit2 = Pet(name='lucy', color='pink', kind='rabbit', age=1)
cat = Pet(name='sandy', color='white', kind='cat', age=6)
cat2 = Pet(name='poker', color='black', kind='cat', age=3)
cat3  = Pet(name='sunshine', color='white', kind='cat', age=2)
owl = Pet(name='cookie', color='brown', kind='owl', age=4)
robot = Robot(name='guy', origin='Spain')

In [12]:
# Create session
session = Session(bind=engine)

In [13]:
# Add objects
# Queue up the queries
session = Session(bind=engine)
session.add(rabbit)
session.add(rabbit2)
session.add(cat)
session.add(cat2)
session.add(cat3)
session.add(owl)
session.add(robot)

In [14]:
# Check queue and see which data is ready to go into the database
session.new

IdentitySet([<__main__.Pet object at 0x00000205AF26B608>, <__main__.Pet object at 0x00000205AF26B5C8>, <__main__.Pet object at 0x00000205AA6EE1C8>, <__main__.Pet object at 0x00000205AF26BEC8>, <__main__.Pet object at 0x00000205AF26B7C8>, <__main__.Pet object at 0x00000205AF276048>, <__main__.Robot object at 0x00000205AF2760C8>])

In [15]:
# commit() flushes whatever remaining changes remain to the database
# and commits the transaction
session.commit()


In [16]:
# Check queue again
session.new

IdentitySet([])

In [17]:
# Check tables
Base.metadata.tables

immutabledict({'pet': Table('pet', MetaData(bind=None), Column('id', Integer(), table=<pet>, primary_key=True, nullable=False), Column('name', String(length=255), table=<pet>), Column('color', String(length=255), table=<pet>), Column('kind', String(length=255), table=<pet>), Column('age', Integer(), table=<pet>), schema=None), 'robot': Table('robot', MetaData(bind=None), Column('id', Integer(), table=<robot>, primary_key=True, nullable=False), Column('name', String(length=255), table=<robot>), Column('origin', String(length=255), table=<robot>), schema=None)})

## Query table

In [18]:
# Query all items
# method 1
pets = session.query(Pet)
for p in pets:
    print(p.name)

mimi
lucy
sandy
poker
sunshine
cookie


In [19]:
# method 2
session.query(Pet.name, Pet.kind, Pet.color, Pet.age).all()

[('mimi', 'rabbit', 'white', 2),
 ('lucy', 'rabbit', 'pink', 1),
 ('sandy', 'cat', 'white', 6),
 ('poker', 'cat', 'black', 3),
 ('sunshine', 'cat', 'white', 2),
 ('cookie', 'owl', 'brown', 4)]

In [20]:
cat_count = session.query(Pet).filter_by(kind='cat').count()
cat_count

3

In [21]:
# More than 1 filter condition

# method 1
white_cat = session.query(Pet).filter_by(kind='cat', color='white')
white_cat.count()

2

In [22]:
# method 2
white_cat = session.query(Pet.name, Pet.color).filter((Pet.kind=='cat') & (Pet.color=='white'))
white_cat.count()

2

In [23]:
type(white_cat)

sqlalchemy.orm.query.Query

In [24]:
for row in white_cat.all():
    print(row)

('sandy', 'white')
('sunshine', 'white')


In [25]:
# How many types of pet
by_kind = session.query(Pet).group_by(Pet.kind)
by_kind.count()

3

## Update data

In [26]:
older_owl = session.query(Pet).filter_by(kind='owl').first()
print(older_owl.age)
older_owl.age += 1
print(older_owl.age)


4
5


In [27]:
# Check queue and see which data is ready to be updated
session.dirty

IdentitySet([<__main__.Pet object at 0x00000205AF276048>])

In [28]:
session.commit()

In [29]:
session.dirty

IdentitySet([])

In [30]:
session.query(Pet.name, Pet.color, Pet.kind, Pet.age).all()

[('mimi', 'white', 'rabbit', 2),
 ('lucy', 'pink', 'rabbit', 1),
 ('sandy', 'white', 'cat', 6),
 ('poker', 'black', 'cat', 3),
 ('sunshine', 'white', 'cat', 2),
 ('cookie', 'brown', 'owl', 5)]

## Delete Data

In [31]:
# Delete one black cat
black_cat = session.query(Pet).filter_by(kind='cat', color='black').one()
black_cat

In [32]:
session.delete(black_cat)

In [33]:
# Check queue and see which data is ready to be deleted
session.deleted

IdentitySet([<__main__.Pet object at 0x00000205AF26BEC8>])

In [34]:
session.commit()

In [35]:
session.deleted

IdentitySet([])

In [36]:
session.query(Pet.name, Pet.color, Pet.kind, Pet.age).all()

[('mimi', 'white', 'rabbit', 2),
 ('lucy', 'pink', 'rabbit', 1),
 ('sandy', 'white', 'cat', 6),
 ('sunshine', 'white', 'cat', 2),
 ('cookie', 'brown', 'owl', 5)]

In [37]:
# Delete all white cats
session.query(Pet).filter_by(kind='cat', color='white').delete()
session.commit()


session.query(Pet.name, Pet.color, Pet.kind, Pet.age).all()

[('mimi', 'white', 'rabbit', 2),
 ('lucy', 'pink', 'rabbit', 1),
 ('cookie', 'brown', 'owl', 5)]

In [38]:
youngest = session.query(Pet).order_by(Pet.age).first()
youngest.name

'lucy'

In [39]:
# Close session
session.close()

## Reflection of existing database

#### To automatically create Object Relationship Mapping classes from an exisiting databse

In [40]:
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [41]:
engine = create_engine(f'sqlite:///Resources/dow.sqlite')
Base = automap_base()

# Use Base class to reflect database tables
Base.prepare(engine, reflect=True)

# Print all classes (tables) mapped to Base
Base.classes.keys()

['dow']

In [42]:
# Assign dow class to Dow
Dow = Base.classes.dow

In [43]:
session = Session(engine)

first_dow = session.query(Dow).first()
first_dow

In [44]:
# Show columns and data in dictionary format
first_dow.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x205af2e7948>,
 'stock': 'AA',
 'open_price': 15.82,
 'id': 1,
 'low_price': 15.78,
 'close_price': 16.42,
 'percent_change': None,
 'quarter': 1,
 'date': '2011-01-07',
 'high_price': 16.72,
 'volume': 239655616}

In [45]:
# Show first 5 close prices only
session.query(Dow.stock, Dow.close_price).limit(5).all()

[('AA', 16.42), ('AA', 15.97), ('AA', 15.79), ('AA', 16.13), ('AA', 17.14)]

In [46]:
# Show first 5 close prices only
for row in session.query(Dow.stock, Dow.close_price).limit(5).all():
    print(row)

('AA', 16.42)
('AA', 15.97)
('AA', 15.79)
('AA', 16.13)
('AA', 17.14)


In [47]:
session.close()

In [48]:
# Inspector Tool
inspector = inspect(engine)

# Get the table names from a database
inspector.get_table_names()

['dow']

In [49]:
# print column names and types
cols = inspector.get_columns('dow')
for col in cols:
    print(col['name'], col['type'])

id INTEGER
quarter INTEGER
stock TEXT
date TEXT
open_price FLOAT
high_price FLOAT
low_price FLOAT
close_price FLOAT
volume INTEGER
percent_change FLOAT
